In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# -------------------------
# Load dataset
# -------------------------
file_path = "mini1.csv"   # adjust path if needed
data = pd.read_csv(file_path)

# -------------------------
# Split comma-separated columns
# -------------------------
def split_and_flatten(df, columns):
    for col in columns:
        split_cols = df[col].astype(str).str.split(",", expand=True)
        split_cols = split_cols.rename(columns=lambda x: f"{col}_{x+1}")
        df = pd.concat([df, split_cols], axis=1)
        df.drop(col, axis=1, inplace=True)
    return df

columns_to_split = ['A', 'B', 'C', 'DR', 'A.1', 'B.1', 'C.1', 'DR.1']
data = split_and_flatten(data, columns_to_split)

# -------------------------
# One-hot encode categoricals
# -------------------------
categorical_columns = [
    col for col in data.columns
    if data[col].dtype == 'object' and col != 'SURVIVAL'
]

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[categorical_columns])
encoded_df = pd.DataFrame(
    encoded_data,
    columns=encoder.get_feature_names_out(categorical_columns)
)

data.drop(categorical_columns, axis=1, inplace=True)
data = pd.concat([data, encoded_df], axis=1)

# -------------------------
# Normalize numerical columns
# -------------------------
numerical_columns = [col for col in data.columns if col not in ['SURVIVAL']]
data[numerical_columns] = data[numerical_columns].apply(
    pd.to_numeric, errors='coerce'
)
data[numerical_columns] = (
    data[numerical_columns] - data[numerical_columns].mean()
) / data[numerical_columns].std()

# -------------------------
# ADD CNN FEATURES (KEY CHANGE)
# -------------------------
NUM_SAMPLES = len(data)
CNN_FEATURE_DIM = 128

# During training → no images → use zeros
cnn_features = np.zeros((NUM_SAMPLES, CNN_FEATURE_DIM))

# -------------------------
# Final ANN input
# -------------------------
X_tabular = data.drop(['SURVIVAL'], axis=1).values
X = np.concatenate([X_tabular, cnn_features], axis=1)
y = data['SURVIVAL'].values

print("Final ANN input shape:", X.shape)  # should be (n, 243)

# -------------------------
# Train-test split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------------
# Define ANN model (UPDATED INPUT SIZE)
# -------------------------
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

# -------------------------
# Train
# -------------------------
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# -------------------------
# Evaluate
# -------------------------
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# -------------------------
# Save model
# -------------------------
model.save("best_model.h5")
print("✅ Hybrid ANN model saved as best_model.h5")


FileNotFoundError: [Errno 2] No such file or directory: 'mini1.csv'

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

# Load the CSV file
file_path = '/content/mini1.csv'
data = pd.read_csv(file_path)

# Function to split comma-separated values and flatten the DataFrame
def split_and_flatten(df, columns):
    for col in columns:
        # Split the column into separate features
        split_cols = df[col].str.split(',', expand=True)
        split_cols = split_cols.rename(columns=lambda x: f"{col}_{x+1}")
        # Concatenate the new columns to the original DataFrame
        df = pd.concat([df, split_cols], axis=1)
        df.drop(col, axis=1, inplace=True)
    return df

# Columns to split
columns_to_split = ['A', 'B', 'C', 'DR', 'A.1', 'B.1', 'C.1', 'DR.1']

# Split and flatten the DataFrame
data = split_and_flatten(data, columns_to_split)

# Identify columns with categorical values
categorical_columns = [col for col in data.columns if data[col].dtype == 'object' and col != 'SURVIVAL']

# Apply one-hot encoding to categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(data[categorical_columns])

# Convert encoded data to DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Drop original categorical columns and concatenate the encoded columns
data.drop(categorical_columns, axis=1, inplace=True)
data = pd.concat([data, encoded_df], axis=1)

# Normalize the numerical columns
numerical_columns = [col for col in data.columns if col not in ['SURVIVAL', 'MLC']]
data[numerical_columns] = data[numerical_columns].apply(pd.to_numeric, errors='coerce')
data[numerical_columns] = (data[numerical_columns] - data[numerical_columns].mean()) / data[numerical_columns].std()

# Separate features and target variable
X = data.drop(['SURVIVAL'], axis=1)
y = data['SURVIVAL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {loss}, Test MAE: {mae}")

# Function to preprocess a single input for prediction
def preprocess_input(input_data, encoder, numerical_columns, categorical_columns):
    # Create a DataFrame for the input data
    input_df = pd.DataFrame([input_data])

    # Split and flatten the DataFrame
    input_df = split_and_flatten(input_df, columns_to_split)

    # Apply one-hot encoding to categorical columns
    encoded_input_data = encoder.transform(input_df[categorical_columns])
    encoded_input_df = pd.DataFrame(encoded_input_data, columns=encoder.get_feature_names_out(categorical_columns))

    # Drop original categorical columns and concatenate the encoded columns
    input_df.drop(categorical_columns, axis=1, inplace=True)
    input_df = pd.concat([input_df, encoded_input_df], axis=1)

    # Normalize the numerical columns
    input_df[numerical_columns] = input_df[numerical_columns].apply(pd.to_numeric, errors='coerce')
    input_df[numerical_columns] = (input_df[numerical_columns] - data[numerical_columns].mean()) / data[numerical_columns].std()

    return input_df

# Example input data
example_input = {
    'A': '3,9',
    'B': '7,40',
    'C': 'w3,-',
    'DR': 'w2,-',
    'A.1': '1,3',
    'B.1': '7,8',
    'C.1': 'w1,w1',
    'DR.1': 'w2,w3',
    'MLC': 0
}

# Preprocess the input data
preprocessed_input = preprocess_input(example_input, encoder, numerical_columns, categorical_columns)

# Make a prediction
prediction = model.predict(preprocessed_input)

print(f"Predicted Survival: {prediction[0][0]}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100
2/2 [==============================] - 1s 213ms/step - loss: 150.3745 - mean_absolute_error: 12.0193 - val_loss: 119.2152 - val_mean_absolute_error: 10.7386
Epoch 2/100
2/2 [==============================] - 0s 33ms/step - loss: 137.1249 - mean_absolute_error: 11.4454 - val_loss: 108.3713 - val_mean_absolute_error: 10.2192
Epoch 3/100
2/2 [==============================] - 0s 32ms/step - loss: 124.8994 - mean_absolute_error: 10.8964 - val_loss: 97.9638 - val_mean_absolute_error: 9.6957
Epoch 4/100
2/2 [==============================] - 0s 31ms/step - loss: 113.3701 - mean_absolute_error: 10.3448 - val_loss: 87.6661 - val_mean_absolute_error: 9.1516
Epoch 5/100
2/2 [==============================] - 0s 50ms/step - loss: 102.1821 - mean_absolute_error: 9.7760 - val_loss: 77.5093 - val_mean_absolute_error: 8.5750
Epoch 6/100
2/2 [==============================] - 0s 38ms/step - loss: 90.3345 - mean_absolute_error: 9.1329 - val_loss: 67.1439 - val_mean_absolute_error: 7.9376
Ep